In [1]:
import pandas as pd
asin = pd.read_csv('ASIN.csv')
print(asin.shape)
asin.head()

(183138, 1)


,asin
0,B016I2TS4W
1,B01N49AI08
2,B01JDPCOHO
3,B01N19U5H5
4,B004GSI2OS


In [2]:
from lxml import html  
import csv,os,json
from exceptions import ValueError
import requests
from time import sleep
 
def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    page = requests.get(url,headers=headers)
    while True:
        sleep(15)
        try:
            doc = html.fromstring(page.content)
            XPATH_title = '//h1[@id="title"]//text()'
            XPATH_formatted_price = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_color = '//span[contains(text(),"selection")/text()'
            XPATH_brand = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_product_type_name = '//div[@id="a-form-label"]//text()'
            XPATH_medium_image_url = '//div[@id="imgTagWrapperID"]//text()'
 
            RAW_title = doc.xpath(XPATH_title)
            RAW_formatted_price = doc.xpath(XPATH_formatted_price)
            RAW_brand = doc.xpath(XPATH_brand)
            RAW_color = doc.xpath(XPATH_color)
            RAw_product_type_name = doc.xpath(XPATH_product_type_name)
            RAW_medium_image_url = doc.xpath(XPATH_medium_image_url)
            
 
            title = ' '.join(''.join(RAW_title).split()) if RAW_title else None
            formatted_price = ' '.join(''.join(RAW_formatted_price).split()).strip() if RAW_formatted_price else None
            brand = ' > '.join([i.strip() for i in RAW_brand]) if RAW_brand else None
            color = ''.join(RAW_color).strip() if RAW_color else None
            product_type_name = ''.join(RAw_product_type_name).strip() if RAw_product_type_name else None
            medium_image_url=''.join(RAW_medium_image_url).strip() if RAW_medium_image_url else None
 
            if page.status_code!=200:
                raise ValueError('captha')
            data = {
                    'asin':i,
                    'brand':brand,
                    'color':color,
                    'medium_image_url':medium_image_url,
                    'product_type_name':product_type_name,
                    'title':title,
                    'formatted_price':formatted_price,     
                    }
 
            return data
        except Exception as e:
            print(e)
 
def ReadAsin():
        extracted_data = []
        for i in asin:
            url = "http://www.amazon.com/dp/"+i
            extracted_data.append(AmzonParser(url))
            sleep(15)
            
        with open("scrapped_data.csv","w") as f:
            w = csv.writer(f,delimiter="\n")
            w.writerow(extracted_data)
        print('done')
    
    
    
if __name__ == "__main__":
    ReadAsin()

done


In [3]:
data=pd.read_csv('scrapped_data.csv')
print(data.shape)
data.head()

(183138, 7)


,asin,brand,color,medium_image_url,product_type_name,title,formatted_price
0,B016I2TS4W,FNC7C,NaN,https://images-na.ssl-images-amazon.com/images...,SHIRT,Minions Como Superheroes Ironman Long Sleeve R...,NaN
1,B01N49AI08,FIG Clothing,NaN,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Izo Tunic,NaN
2,B01JDPCOHO,FIG Clothing,NaN,https://images-na.ssl-images-amazon.com/images...,SHIRT,FIG Clothing Womens Won Top,NaN
3,B01N19U5H5,Focal18,NaN,https://images-na.ssl-images-amazon.com/images...,SHIRT,Focal18 Sailor Collar Bubble Sleeve Blouse Shi...,NaN
4,B004GSI2OS,FeatherLite,Onyx Black/ Stone,https://images-na.ssl-images-amazon.com/images...,SHIRT,Featherlite Ladies' Long Sleeve Stain Resistan...,$26.26
